In [1]:
# CHALLENGE PROBLEM: 
#
# Use your check_sudoku function as the basis for solve_sudoku(): a
# function that takes a partially-completed Sudoku grid and replaces
# each 0 cell with an integer in the range 1..9 in such a way that the
# final grid is valid.
#
# There are many ways to cleverly solve a partially-completed Sudoku
# puzzle, but a brute-force recursive solution with backtracking is a
# perfectly good option. The solver should return None for broken
# input, False for inputs that have no valid solutions, and a valid
# 9x9 Sudoku grid containing no 0 elements otherwise. In general, a
# partially-completed Sudoku grid does not have a unique solution. You
# should just return some member of the set of solutions.
#
# A solve_sudoku() in this style can be implemented in about 16 lines
# without making any particular effort to write concise code.

# solve_sudoku should return None
ill_formed = [[5,3,4,6,7,8,9,1,2],
              [6,7,2,1,9,5,3,4,8],
              [1,9,8,3,4,2,5,6,7],
              [8,5,9,7,6,1,4,2,3],
              [4,2,6,8,5,3,7,9],  # <---
              [7,1,3,9,2,4,8,5,6],
              [9,6,1,5,3,7,2,8,4],
              [2,8,7,4,1,9,6,3,5],
              [3,4,5,2,8,6,1,7,9]]

# solve_sudoku should return valid unchanged
valid = [[5,3,4,6,7,8,9,1,2],
         [6,7,2,1,9,5,3,4,8],
         [1,9,8,3,4,2,5,6,7],
         [8,5,9,7,6,1,4,2,3],
         [4,2,6,8,5,3,7,9,1],
         [7,1,3,9,2,4,8,5,6],
         [9,6,1,5,3,7,2,8,4],
         [2,8,7,4,1,9,6,3,5],
         [3,4,5,2,8,6,1,7,9]]

# solve_sudoku should return False
invalid = [[5,3,4,6,7,8,9,1,2],
           [6,7,2,1,9,5,3,4,8],
           [1,9,8,3,8,2,5,6,7],   # <------ 8 appears two times
           [8,5,9,7,6,1,4,2,3],
           [4,2,6,8,5,3,7,9,1],
           [7,1,3,9,2,4,8,5,6],
           [9,6,1,5,3,7,2,8,4],
           [2,8,7,4,1,9,6,3,5],
           [3,4,5,2,8,6,1,7,9]]

# solve_sudoku should return a 
# sudoku grid which passes a 
# sudoku checker. There may be
# multiple correct grids which 
# can be made from this starting 
# grid.
easy = [[2,9,0,0,0,0,0,7,0],
        [3,0,6,0,0,8,4,0,0],
        [8,0,0,0,4,0,0,0,2],
        [0,2,0,0,3,1,0,0,7],
        [0,0,0,0,8,0,0,0,0],
        [1,0,0,9,5,0,0,6,0],
        [7,0,0,0,9,0,0,0,1],
        [0,0,1,2,0,0,3,0,6],
        [0,3,0,0,0,0,0,5,9]]

# Note: this may timeout 
# in the Udacity IDE! Try running 
# it locally if you'd like to test 
# your solution with it.

hard = [[1,0,0,0,0,7,0,9,0],
        [0,3,0,0,2,0,0,0,8],
        [0,0,9,6,0,0,5,0,0],
        [0,0,5,3,0,0,9,0,0],
        [0,1,0,0,8,0,0,0,2],
        [6,0,0,0,0,4,0,0,0],
        [3,0,0,0,0,0,0,1,0],
        [0,4,0,0,0,0,0,0,7],
        [0,0,7,0,0,0,3,0,0]]

import copy

def columns(grid):
    '''Helper function that transposes grid and returns a list of columns'''
    return [[row[i] for row in grid] for i in range(9)]

def subgrids(grid):
    '''Helper function that returns a list of the nine 3x3 sub-grids in horizontal ordering'''
    return [[grid[i+m][j+k] for m in range(3) for k in range(3)] for i in range(0, 7, 3) for j in range(0, 7, 3)]

def check_sudoku(grid):
    """Sudoku puzzle validity checker function. 
    Returns None if input is not a 9x9 grid of the form [[int,..],... []] where int is an integer from 0 to 9. 
    Returns False if the grid is not a valid Sudoku puzzle and True otherwise.
    Zeros are accepted as representing unsolved cells."""
    # input validity checks
    if not isinstance(grid, list):
        return None
    elif not len(grid) == 9:
        return None
    elif not all([isinstance(row, list) for row in grid]):
        return None
    elif not all([len(row) == 9 for row in grid]):
        return None
    elif not all([isinstance(char, int) for row in grid for char in row]):
        return None
    elif not all([(0 <= char <= 9) for row in grid for char in row]):
        return None
    # create list of checks for rows, columns and sub_grinds
    checks = [grid, columns(grid), subgrids(grid)]
    for check in checks:
        if not all([row.count(num) <= 1 for row in check for num in range(1, 10)]):
            return False
    return True

def create_puzzle(grid):
    """Helper function that takes an unsolved grid and replaces zeros with the set {1, ..., 9}.
    Solved cells are transformed to one element sets."""
    puzzle = [[], [], [], [], [], [], [], [], []]
    for counter, row in enumerate(grid):
        for elem in row:       
            if elem == 0:
                puzzle[counter].append(set(range(1, 10)))
            else:
                puzzle[counter].append(set([elem]))
    return puzzle

def convert_to_grid(puzzle):
    """Helper function to convert a sudoku puuzle of the format [[{}, ...], ... , [{}, ...]]
    back to the original format of [[int, ..., int], ..., [int, ..., int]]."""
    grid = [[], [], [], [], [], [], [], [], []]
    for i, row in enumerate(puzzle):
        for j, elem in enumerate(row):
            if len(elem) == 1:
                grid[i].append(next(iter(elem)))
            else:
                grid[i].append(0)
    return grid

def solved_cells(puzzle):
    """Helper function that counts solved cell in a sodoku grid, where each cell is a set"""
    return sum([1 for row in puzzle for elem in row if len(elem) == 1])

def first_stage_solve(puzzle):
    """Helper function that eliminates possible values from unsolved cells, based on existing solved
    cells in corresponding row, column and subgrid and returns the result."""
    result = [[], [], [], [], [], [], [], [], []] 
    for i, row in enumerate(puzzle):
         for j, cell in enumerate(row):
            if len(cell) > 1:
                # construct a set of solved cell values in row i
                solved_row_values = set([]).union(*[elem for elem in row if len(elem) == 1])
                # construct a set of solved cell values in column j
                solved_column_values = set([]).union(*[elem for elem in columns(puzzle)[j] if len(elem) == 1])
                # construct a set of solved cell values in corresponding subgrid
                solved_subgrid_values = set([]).union(*[elem for elem in subgrids(puzzle)[3*(i//3) + j//3] if len(elem) == 1])
                # construct the set of not permitted values for cell
                not_permitted = solved_row_values | solved_column_values | solved_subgrid_values
                # eliminate not permitted values and append to result
                result[i].append(cell.difference(not_permitted))
            else:
                result[i].append(cell)
    return result

def second_stage_solve(puzzle):
    """Helper function that solves unsolved cells, by checking for uniqueness of allowed values
    for the cell in corresponding row and column and returns the result."""
    # check rows
    for i, row in enumerate(puzzle):
         for j, cell in enumerate(row):
            if len(cell) > 1:
                other_cell_values = set([]).union(*[elem for counter, elem in enumerate(row) if counter != j]) 
                for value in cell:
                    if not value in other_cell_values:
                        puzzle[i][j] = set([value])
                        break
    # check columns
    for j, column in enumerate(columns(puzzle)):
         for i, cell in enumerate(column):
            if len(cell) > 1:
                other_cell_values = set([]).union(*[elem for counter, elem in enumerate(column) if counter != i]) 
                for value in cell:
                    if not value in other_cell_values:
                        puzzle[i][j] = set([value])
                        break
    return puzzle

def pairs_of_two(row_column_subgrid):
    """Helper function that returns the positions of the first pair of identical 2-sets found in input
    row, column or subgrid."""
    for elem in row_column_subgrid:
        if len(elem) == 2 and row_column_subgrid.count(elem) == 2:
            i = row_column_subgrid.index(elem)
            j = i + 1 + row_column_subgrid[i+1: ].index(elem)
            return (i, j)
    else:
        return None

def third_stage_solve(puzzle):
    """Helper function that scans each row, column and subgrid for pairs of cells with same 2-set
    of possible solutions and eliminates these values from remaining cells of the corresponding
    row, column or subgrid.""" 
    # scan rows
    for i, row in enumerate(puzzle):
        pair_positions = pairs_of_two(row)
        if pair_positions:
            for j, cell in enumerate(row):
                if j not in pair_positions:
                    puzzle[i][j] = cell.difference(row[pair_positions[0]])
    # scan columns
    for j, column in enumerate(columns(puzzle)):
        pair_positions = pairs_of_two(column)
        if pair_positions:
            for i, cell in enumerate(column):
                if i not in pair_positions:
                    puzzle[i][j] = cell.difference(column[pair_positions[0]])
    # scan subgrids
    for i, subgrid in enumerate(subgrids(puzzle)):
        pair_positions = pairs_of_two(subgrid)
        if pair_positions:
            for j, cell in enumerate(subgrid):
                if j not in pair_positions:
                    puzzle[3*(i//3) + j//3][3*(i%3) + j%3] = cell.difference(subgrid[pair_positions[0]])
    return puzzle

def logic_solve(puzzle):
    """Helper function that attempts to solve the puzzle using logic"""
    while True:
        solved_cells_at_start = solved_cells(puzzle)
        puzzle = third_stage_solve(second_stage_solve(first_stage_solve(puzzle)))
        solved_cells_at_end = solved_cells(puzzle)
        if solved_cells_at_end == solved_cells_at_start:
            return puzzle  

def backtracking_solve(puzzle):
    """Helper function that solves the puzzle recursively using backtracking."""
    for i, row in enumerate(puzzle):
        for j, cell in enumerate(row):
            if len(cell) > 1:
                for value in cell:
                    # puzzle is a compound list, so a slice copy wouldn't work 
                    # here as the original puzzle would be affected
                    possible_solution = copy.deepcopy(puzzle)
                    possible_solution[i][j] = set([value])
                    possible_solution = logic_solve(possible_solution)
                    if solved_cells(possible_solution) == 81:
                        if check_sudoku(convert_to_grid(possible_solution)):
                            return possible_solution
                        else:
                            continue
                    elif not all([len(elem) > 0 for row in possible_solution for elem in row]):
                        continue
                    else:
                        possible_solution = backtracking_solve(possible_solution)
                        if possible_solution is not None:
                            return possible_solution
                        else:
                            continue
                # Possible values for cell exhausted. Either we backtrack or the cell examined
                # was the last one and the puzzle has no solution
                return None
                    
def solve_sudoku(grid):
    if check_sudoku(grid) is None:
        return None
    elif not check_sudoku(grid):
        return False
    elif all([elem != 0 for row in grid for elem in row]):
        return grid
       
    puzzle = logic_solve(create_puzzle(grid))
    if solved_cells(puzzle) == 81:
        result = convert_to_grid(puzzle)
        assert check_sudoku(result)
        return result
    elif not all([len(elem) > 0 for row in puzzle for elem in row]):
        return False
    else:
        puzzle = backtracking_solve(puzzle)
        if puzzle is None:
            return False
        else: 
            result = convert_to_grid(puzzle)
            assert check_sudoku(result)
            return result

# print solve_sudoku(ill_formed) # --> None
# print solve_sudoku(valid)      # --> True
# print solve_sudoku(invalid)    # --> False

test1 = [[8,0,0,0,0,0,0,0,0],
        [0,0,3,6,0,0,0,0,0],
        [0,7,0,0,9,0,2,0,0],
        [0,5,0,0,0,7,0,0,0],
        [0,0,0,0,4,5,7,0,0],
        [0,0,0,1,0,0,0,3,0],
        [0,0,1,0,0,0,0,6,8],
        [0,0,8,5,0,0,0,1,0],
        [0,9,0,0,0,0,4,0,0]]

test2 = [[7,0,0,2,0,0,0,0,0],
        [8,0,0,0,0,0,0,4,0],
        [0,0,0,1,0,0,0,0,0],
        [0,1,6,5,0,0,2,0,0],
        [0,0,0,0,0,0,0,7,0],
        [0,2,0,0,0,0,0,0,0],
        [0,0,0,0,0,0,6,0,1],
        [3,0,0,0,4,0,0,0,0],
        [0,5,0,0,8,0,0,0,0]]


for row in solve_sudoku(easy):       # --> True
    print row
            
print ''
for row in solve_sudoku(test1):       # --> True
    print row

SyntaxError: invalid syntax (<ipython-input-1-4241054eb76d>, line 287)